### 获取词空间向量的方法

#### 传统方法

传统求取word的空间向量表征:<br>
(1) LSA 将词和文档映射到潜在语义空间，从而去除了原始
向量空间中的一些“噪音”，但它无法保存词与词之间的linear regularities； <br>
(2) LDA 是一个三层贝叶斯概率模型，包含词、主题和文档三层结构。文档到主题服从Dirichlet分布，主
题到词服从多项式分布， 但是只要训练数据大了， 计算量就一下飚了。

#### 基于深度学习的方法

基于神经网络的词语向量表征方法在2003中就有提出，名为NNLM，它是一个前向网络，同时学习词语表
征和一个统计语言模型（后面具体讲）。<br>
在Mikolov的硕士论文和他在ICASSP 2009上发表的文章中， 用一个单隐层网络训练词语表征， 然后
将这个表征作为NNLM的输入进行训练。 Word2vec是训练词语表征工作的一个拓展。


### 对语料进行预处理

预处理博文：https://www.cnblogs.com/pinard/p/6744056.html

#### 加载jieba软件

In [3]:
import jieba
import jieba.analyse

#### 添加专用名词（电视剧中人名）

In [4]:
jieba.suggest_freq('沙瑞金', True)
jieba.suggest_freq('田国富', True)
jieba.suggest_freq('高育良', True)
jieba.suggest_freq('侯亮平', True)
jieba.suggest_freq('钟小艾', True)
jieba.suggest_freq('陈岩石', True)
jieba.suggest_freq('欧阳菁', True)
jieba.suggest_freq('易学习', True)
jieba.suggest_freq('王大路', True)
jieba.suggest_freq('蔡成功', True)
jieba.suggest_freq('孙连城', True)
jieba.suggest_freq('季昌明', True)
jieba.suggest_freq('丁义珍', True)
jieba.suggest_freq('郑西坡', True)
jieba.suggest_freq('赵东来', True)
jieba.suggest_freq('高小琴', True)
jieba.suggest_freq('赵瑞龙', True)
jieba.suggest_freq('林华华', True)
jieba.suggest_freq('陆亦可', True)
jieba.suggest_freq('刘新建', True)
jieba.suggest_freq('刘庆祝', True)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Neo\AppData\Local\Temp\jieba.cache
Loading model cost 0.699 seconds.
Prefix dict has been built succesfully.


1

#### 处理中文编码问题

#### 中文分词

简单的IO读取介绍：https://www.cnblogs.com/ymjyqsx/p/6554817.html

In [5]:
with open("./data02.txt", encoding="utf-8") as f: #也可以加 encoding="utf-8" 直接读取为中文
    document = f.read()  
    #doc = document.decode("utf-8")  # 字符文本和二进制文本的转换 encode（）和decode（）
    document_cut = jieba.cut(document)   # 产生的是一个document_cut生成器，生成的是字符类型的词群

#### 去停用词

In [6]:
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, "r", encoding="utf-8").readlines()]
    return stopwords

In [7]:
result = ""
for word in document_cut:
    if word not in stopwordslist("./stop_words.txt"):
        if word != "\t" and word !="\n":
            result += word
            result += " "
result = result.encode("utf-8")
with open("./segment02.txt", "wb") as f:
    f.write(result)

### word2vec的使用

#### gensim-Word2vec架构图

In [9]:
%%html
<img src="./gensim-Word2vec架构.jpg" width="100%">

gensim中word2vec官网：<br>
https://radimrehurek.com/gensim/models/word2vec.html<br>
word2vec.Word2Vec网页：<br>
https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec

#### 训练模型

In [10]:
import logging
import os
from gensim.models import word2vec
#logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)
sentences = word2vec.LineSentence("./segment01.txt")
model = word2vec.Word2Vec(sentences, sg=0, hs=1, min_count=1, window=3, size=100, workers=3)
# 参数解释
# sentences：模型语料（大语料处理方法见官方此处文档）
# corpus——file：语料路径（可代替sentences参数）
# size： 词向量维度
# window：窗口距离（单边最大距离）
# min_count： 低频次过滤阈值
# workers： 线程数量
# sg: 1:skip-gram  0:CBOW
# hs: 1:hierarchical softmax  0:negative sampling
# negative(int)： 负采样中负样本个数
# seed：采用负采样时，初始向量使用上下文词向量加总（0）还是加总后的均值（1）
# alpha: 学习率
# min_alpha： 最小学习率
# seed：随机化词向量时种子随机数
# max_vocab_size: 基于RAM容量的词表最大数
# max_final_vocab： 最终词表最大数量
# sampling：高频词的控制选择阈值
# iter：样本集迭代次数
# sorted_vocab： 给词典词与排序
# batch_words：小批量词训练
# compute_loss： 最终损失函数值，供查询
# ...

#### 保存和提取模型

##### 存储模型

In [11]:
# 方式一
model.save("./model01_save.txt")  # 存储为二进制形式
# 方式二
model.wv.save_word2vec_format("./model01_format_save.txt", binary = False)
#相同点：都可以复用，即载入之后可以得到对应单词的词向量；
#不同点：save保存的模型，载入之后可以继续在此基础上接着训练，而format_save保存
#       的模型不能，但有个好处就是如果s设置binary=False，则保存之后可以直接用
#       记事本打开看参数；

##### 加载模型

In [12]:
import gensim
new_model = gensim.models.Word2Vec.load("./model01_save.txt")

##### 查看模型参数

In [13]:
# 方式一：通过model_wv.save_word2vec_format(..., binary=False)保存文件，从记事本打开，
#        不过文件一般比价大，不要通过记事本，通过代码编辑器打开比较好
# 方式二：通过加载模型的函数和属性查看
print(new_model.wv.word_vec("侯亮平"))
print()
print(type(new_model.wv.word_vec("侯亮平")))
print()
print(new_model.wv.vocab["侯亮平"])
print()
#print(new_model.wv.vocab)  # 这是一个词典，体积太大，需要看的时候再加载

[-0.34591576  0.3888605   0.3814108   0.7239727  -0.7889621   0.41575894
  0.5826813  -0.23850438  0.3892129   0.2742579   0.7946546  -0.11903445
  0.42770007  0.42080036  0.09306762 -0.7537523  -0.5480356  -0.396157
 -0.5726908   0.14850919  0.2056894   0.3152269   1.2542996  -0.11404041
  0.72683215  0.61925155 -0.4399952   0.3263669  -0.1389091   0.35586643
 -0.31470793  1.3053495  -0.21782136  0.2372876   0.7229889   0.2620929
  0.15740258 -0.07465599  0.98209584  0.07359022  0.27534112  0.44218102
  0.5375589   0.36040753 -0.30162454 -0.1746489  -0.00399005  0.40247214
 -0.2793665   0.12225086 -1.251732    0.19966249  0.7563926   0.26516706
 -0.13449042 -0.87608045  0.8644344  -0.0283279   0.23773886  0.41984966
 -0.11118336 -0.03967812 -1.278302   -0.48248178  0.27822348 -0.48317397
  0.20735563  0.7640554  -0.5119091   1.1886377   0.18875057  0.23382966
  0.07259258  0.57353485 -0.45584524 -0.08017226  0.2507858   1.0381241
  0.23747715 -0.05242451  0.22965065  0.18194777  0.488

##### 增量训练

In [14]:
print("语料数：",new_model.corpus_count)
print("词表长度：",len(new_model.wv.vocab)) 

语料数： 7
词表长度： 10667


In [15]:
new_model.build_vocab(sentences = "./segment02.txt", update=True)
new_model.train(sentences= "./segment02.txt", \
                total_examples=new_model.corpus_count, epochs=new_model.epochs)

new_model.save("./model02_save.txt")
new_model.wv.save_word2vec_format("./model02_format_save.txt", binary=False)

print("以下是两次训练各自的总和")
print("语料数：",new_model.corpus_count)
print("词表长度：",len(new_model.wv.vocab)) # new_model.wv.vocab是一个字典

以下是两次训练各自的总和
语料数： 15
词表长度： 10675


### word2vec的功能

#### 功能01：找出与指定词最相近的n个词语

In [16]:
req_count = 5
for key in new_model.wv.similar_by_word("沙瑞金", topn=100):
    if len(key[0]) == 3:
        req_count -= 1
        print(key[0], key[1])
        if req_count == 0:
            break

陈岩石 0.9957113265991211
祁同伟 0.995481550693512
赵立春 0.9926450252532959
欧阳菁 0.9921784400939941
郑西坡 0.9916276931762695


In [17]:
new_model.wv.most_similar(["沙瑞金"], topn=5)

[('人', 0.9975489377975464),
 ('又', 0.9971535205841064),
 ('进来', 0.9969267845153809),
 ('陈岩石', 0.9957113265991211),
 ('，', 0.9956244230270386)]

#### 功能02：计算两个词之间的相似度（距离）

In [18]:
# 计算相似度
print(new_model.wv.similarity("沙瑞金", "高育良"))
print(new_model.wv.similarity("李达康", "王大路"))
# 计算距离
print(new_model.wv.distance("沙瑞金", "高育良"))
print(new_model.wv.distance("李达康", "王大路"))
# 支持词语的加减运算
#print(new_model.wv.most_similar(["woman", "king"], negative=["man"], topn=2))

0.9827868
0.99228823
0.01721322536468506
0.00771176815032959


#### 功能03：计算两个集合之间的余弦似度

In [76]:
list1 = ['我','走','我','学校'] 
list2 = ['我','去','家'] 
list_sim1 = new_model.wv.n_similarity(list1,list2)
print(list_sim1)

0.99887896


#### 功能04：找出指定词语中不属于同一类的词

In [20]:
print(model.wv.doesnt_match("沙瑞金 季昌明 高育良".split()))

沙瑞金


### 使用已经训练好的大规模的词向量模型

已经训练好的中文词向量：https://github.com/Embedding/Chinese-Word-Vectors

#### 加载预训练词向量

In [68]:
#big_model = gensim.models.KeyedVectors.load_word2vec_format("./renminribao.word")  # 太大，谨慎执行

In [67]:
# 使用
print(big_model.most_similar(["女士", "国王"], negative=["男士"], topn=1))
print(big_model.doesnt_match("猪肉 手 耳朵 屁股".split()))

[('王后', 0.6075538396835327)]
猪肉


#### 自定义函数查看模型（参数列表）结构

In [73]:
import numpy as np
def read_vectors(path, topn):  # read top n word vectors, i.e. top is 10000
    lines_num, dim = 0, 0
    vectors = {}
    iw = []
    wi = {}
    with open(path, encoding='utf-8', errors='ignore') as f:
        first_line = True
        for line in f:
            if first_line:
                first_line = False
                dim = int(line.rstrip().split()[1])
                continue
            lines_num += 1
            tokens = line.rstrip().split(' ')
            vectors[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])
            iw.append(tokens[0])
            if topn != 0 and lines_num >= topn:
                break
    for i, w in enumerate(iw):
        wi[w] = i
    return vectors, iw, wi, dim

In [74]:
vectors, iw, wi, dim = read_vectors("./renminribao.word", topn=2)
print(iw)
print(wi)
print(dim)
print(vectors)

['，', '的', '。']
{'，': 0, '的': 1, '。': 2}
300
{'，': array([ 1.16763e-01, -8.22600e-02, -6.70780e-02, -2.98830e-02,
       -7.58870e-02, -5.12860e-02,  7.95900e-03,  2.24768e-01,
        2.52038e-01,  2.08444e-01,  1.76290e-02, -1.04018e-01,
       -9.49590e-02,  9.88760e-02, -6.56870e-02,  1.84916e-01,
       -1.27854e-01,  2.48928e-01,  1.04157e-01, -1.48353e-01,
        7.78600e-02, -7.35400e-03, -1.27350e-01, -1.64475e-01,
       -1.80341e-01, -3.58898e-01,  7.81460e-02, -6.76730e-02,
       -5.55760e-02,  2.97150e-02,  1.23122e-01, -1.21995e-01,
       -5.19930e-02, -4.87580e-02, -1.35541e-01,  4.32560e-02,
       -1.83343e-01, -8.06730e-02, -7.59490e-02,  6.18350e-02,
        8.13340e-02,  1.03449e-01,  1.98980e-02,  7.68360e-02,
        8.55830e-02, -7.02630e-02, -3.24490e-02,  1.85880e-02,
        3.36520e-02,  1.54840e-02,  5.54510e-02, -4.56550e-02,
       -1.36057e-01, -6.06910e-02,  7.21270e-02,  3.19910e-02,
        1.88020e-01,  2.07825e-01,  8.17520e-02,  8.37360e-02,
    

### 其他比较重要的词向量训练软件

#### GloVe

#### fastext